In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..', 'src')))
from funkwpap import *
import sympy as sp, pandas as pd, numpy as np, tqdm, time, sys, matplotlib.pyplot as plt, tqdm
from statistics import mean
from scipy.optimize import fsolve
from random import random


In [2]:
x = sp.symbols('x')
cap = 2000 # total emission cap
Regulator25 = Regulator("test1", permit_price = 2.4079, emission_cap = cap)
sector1 = Sector('cement', price_demand_function= 100 - 0.1*x, free_emission_multiplier= 0, regulator= Regulator25)
sector2 = Sector('steel', price_demand_function=150 - 0.1*x, free_emission_multiplier= 0, regulator= Regulator25)
sector3 = Sector('paper', price_demand_function= 200 - 0.02*x**1.5, free_emission_multiplier= 0, regulator= Regulator25)
country1 = Country('DE', 1, regulator= Regulator25)
country2 = Country('FI', 0.5, regulator= Regulator25)
country3 = Country('GR', size= 0.1, regulator= Regulator25)

# Create Firms using objects
firm1 = Firm('firm1', 1, 1, x*0, 2*x+ 2*x**2 + 2*x**3 , 0, 0, 0, regulator= Regulator25)
firm2 = Firm('firm2', 1, 2, x*0, 3*x+ 3*x**2 + 2*x**3, 0, 0, 0, regulator= Regulator25)
firm3 = Firm('firm3', 1, 3, x*0, 1*x+ 3*x**2 + 2*x**3 , 0, 0, 0, regulator= Regulator25)
firm4 = Firm('firm4', 2, 1, x*0, 7*x+ 5*x**2 + 3*x**3 , 0, 0, 0, regulator= Regulator25)
firm5 = Firm('firm5', 2, 2, x*0, 1*x+ 6*x**2 + 2*x**3 , 0, 0, 0, regulator= Regulator25)
firm6 = Firm('firm6', 2, 3, x*0, 2*x+ 7*x**2 + 3*x**3 , 0, 0, 0, regulator= Regulator25)
firm7 = Firm('firm7', 3, 1, x*0, 3*x+ 8*x**2 + 4*x**3 , 0, 0, 0, regulator= Regulator25)
firm8 = Firm('firm8', 3, 2, x*0, 4*x+ 9*x**2 + 10*x**3 , 0, 0, 0, regulator= Regulator25)
firm9 = Firm('firm9', 3, 3, x*0, 5*x+ 10*x**2 + 11*x**3 , 0, 0, 0, regulator= Regulator25)



In [3]:
Regulator25.BAU_calculator()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-14


In [ ]:
# Define the percentages of BAU emissions
do_all_get_free = [False, True]
Regulator25.emission_cap = Regulator25.BAU_emissions * 0.8
# Initialize dictionaries to store results for each percentage
results = {do: {
    'profits1': [], 'profits2': [], 'abatements1': [], 'abatements2': [],
    'permits1': [], 'permits2': [], 'trades1': [], 'trades2': [],
    'revenews1': [], 'revenews2': [], 'permit_price': [],
    'outputs1': [], 'outputs2': [], 'emissions1': [], 'emissions2': []
} for do in do_all_get_free}

# Loop over each percentage
for do in do_all_get_free:
    Regulator25.emission_cap = Regulator25.BAU_emissions * 0.8
    frees = np.linspace(0, 1, 100)

    for free in tqdm.tqdm(frees):
        sector1.free_emission_multiplier = free
        if do:
            sector2.free_emission_multiplier = free*0.5
            sector3.free_emission_multiplier = free*0.5
        # Regulator25.optimization_concave_formulation_ab()
        Regulator25.optimization_with_least_squares_ab()
        print(f"{free}: {Regulator25.equilibrium_tester(output=True)}")

        for firm in Regulator25.firm_registry.values():
            firm.calculate_profit_components()

        results[do]['profits1'].append(mean(float(i.profit) for i in sector1.firms))
        results[do]['profits2'].append(mean(float(i.profit) for i in sector2.firms + sector3.firms))
        results[do]['abatements1'].append(mean(float(i.abatement) for i in sector1.firms))
        results[do]['abatements2'].append(mean(float(i.abatement) for i in sector2.firms + sector3.firms))
        results[do]['permits1'].append(mean(float(i.permits_used) for i in sector1.firms))
        results[do]['permits2'].append(mean(float(i.permits_used) for i in sector2.firms + sector3.firms))
        results[do]['trades1'].append(mean(float(i.permits_costs) for i in sector1.firms))
        results[do]['trades2'].append(mean(float(i.permits_costs) for i in sector2.firms + sector3.firms))
        results[do]['revenews1'].append(mean(float(i.sales) for i in sector1.firms))
        results[do]['revenews2'].append(mean(float(i.sales) for i in sector2.firms + sector3.firms))
        results[do]['permit_price'].append(Regulator25.permit_price)
        results[do]['outputs1'].append(mean(float(i.actual_output) for i in sector1.firms))
        results[do]['outputs2'].append(mean(float(i.actual_output) for i in sector2.firms + sector3.firms))
        results[do]['emissions1'].append(mean(float(i.emission) for i in sector1.firms))
        results[do]['emissions2'].append(mean(float(i.emission) for i in sector2.firms + sector3.firms))



  0%|          | 0/100 [00:00<?, ?it/s]

Optimal solution found


  1%|          | 1/100 [00:06<11:18,  6.86s/it]

Firm       | FOC 1        | FOC 2         | SOC 1         | SOC 2        | Hessian      | Status
-------------------------------------------------------------------------------------
Worst      |   +0.0491 ⚠️ |  +1.86e-4 ✔️ |  -24.6327 ✔️ |  -24.8327 ✔️ |   +4.9265 ✔️ | 4/5
0.0: (False, 0.0490628044262547)
Optimal solution found


  2%|▏         | 2/100 [00:12<10:12,  6.25s/it]

...

Firm       | FOC 1        | FOC 2         | SOC 1         | SOC 2        | Hessian      | Status
-------------------------------------------------------------------------------------
Worst      |   +0.0511 ⚠️ |  +6.35e-4 ✔️ |  -32.1544 ✔️ |  -32.3544 ✔️ |   +6.4309 ✔️ | 4/5
0.8787878787878789: (False, 0.0511472405343625)
Optimal solution found


 89%|████████▉ | 89/100 [12:15<01:35,  8.66s/it]

Firm       | FOC 1        | FOC 2         | SOC 1         | SOC 2        | Hessian      | Status
-------------------------------------------------------------------------------------
Worst      |   +0.0714 ⚠️ |  +2.48e-6 ✔️ |  -32.2711 ✔️ |  -32.4711 ✔️ |   +6.4542 ✔️ | 4/5
0.888888888888889: (False, 0.0714360036189134)
Optimal solution found


 90%|█████████ | 90/100 [12:22<01:21,  8.17s/it]

Firm       | FOC 1        | FOC 2         | SOC 1         | SOC 2        | Hessian      | Status
-------------------------------------------------------------------------------------
Worst      |   +0.2273 ⚠️ |  +4.58e-6 ✔️ |  -32.3665 ✔️ |  -32.5665 ✔️ |   +6.4733 ✔️ | 4/5
0.8989898989898991: (False, 0.227288397540217)


In [ ]:
# Define colors and transparency levels
colors = {'trades1': '#1f77b4', 'trades2': '#ff7f0e'}


# Define a function to plot the metrics
def plot_metric(metric, ylabel, title):
    plt.figure(figsize=(10, 6))
    plt.plot(frees, results[True][f'{metric}1'], label=f'{metric.capitalize()}1 eveyone gets free', color=colors['trades1'])
    plt.plot(frees, results[True][f'{metric}2'], label=f'{metric.capitalize()}2 eveyone gets free', color=colors['trades2'])
    plt.plot(frees, results[False][f'{metric}1'], label=f'{metric.capitalize()}1 only sector1', color=colors['trades1'], linestyle='--')
    plt.plot(frees, results[False][f'{metric}2'], label=f'{metric.capitalize()}2 only sector1', color=colors['trades2'], linestyle='--')
    plt.xlabel('Free')
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend()
    plt.grid()
    plt.show()

# Plot outputs
plot_metric('outputs', 'Output', 'Output (q) vs Free')

# Plot emissions
plot_metric('emissions', 'Emission', 'Emission (x) vs Free')

# Plot permit prices
plt.figure(figsize=(10, 6))
plt.plot(frees, results[True]['permit_price'], label=f'Permit Price eveyone gets free', color=colors['trades1'])
plt.plot(frees, results[False]['permit_price'], label=f'Permit Price only sector1', color=colors['trades1'], linestyle='--')
plt.xlabel('Free')
plt.ylabel('Permit Price')
plt.title('Permit Price vs Free')
plt.legend()
plt.grid()
plt.show()

# Plot trades
plot_metric('trades', 'Trade', 'Money spent on permits vs Free')

# Plot revenues
plot_metric('revenews', 'Revenue', 'Sales vs Free')



# Plot profits
plot_metric('profits', 'Profit', 'Profits vs Free')

# Plot abatements
plot_metric('abatements', 'Abatement', 'Abatements vs Free')

# Plot permits used
plot_metric('permits', 'Permits Used', 'Permits Used vs Free')

